In [1]:
import json
import requests
import os
from datetime import datetime
import zipfile
import tempfile
from collections import defaultdict
import time

In [2]:
JSON_FILE = "./memories_history.json"

In [3]:
# Output folder
output_dir = "snapchat_downloads_renamed"
os.makedirs(output_dir, exist_ok=True)

In [4]:
# Load JSON
with open(JSON_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

In [5]:
memories = data['Saved Media']

In [6]:
def extension_from_headers(headers, media_type):
    content_type = headers.get("Content-Type", "").lower()

    if "video/mp4" in content_type:
        return ".mp4"
    if "image/jpeg" in content_type:
        return ".jpg"
    if "image/png" in content_type:
        return ".png"
    if "image/webp" in content_type:
        return ".webp"
    if "application/zip" in content_type:
        return ".zip"

    # fallback if Snapchat does something weird
    if media_type == "Video":
        return ".mp4"
    if media_type == "Image":
        return ".jpg"

    return ""
    
def safe_filename(s):
    return "".join(c for c in s if c.isalnum() or c in "._- ").rstrip()

def download_file(url, out_path, cookies=None):
    r = requests.get(url, stream=True, cookies=cookies)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return r.headers.get("content-type")

In [9]:
for item in memories:
    url = item["Media Download Url"]
    media_type = item["Media Type"]

    # clean date for filename
    date = datetime.strptime(item["Date"], "%Y-%m-%d %H:%M:%S UTC")
    base_name = date.strftime("%Y-%m-%d_%H%M%S")
    
    # not downloading the same files we already downloaded before failure
#     if base_name in existing_files:
#         continue

    try:
        response = requests.get(url, allow_redirects=True)
        response.raise_for_status()
    except:
        if response.status_code >= 500:
            time.sleep(5)
            try:
                response = requests.get(url, allow_redirects=True)
                response.raise_for_status()
            except:
                print("Could not save: ", filename)
                continue
            

    ext = extension_from_headers(response.headers, media_type)
    filename = os.path.join(output_dir, base_name + ext)
    
    with open(filename, "wb") as f:
        f.write(response.content)

    print("Saved:", filename)


Saved: snapchat_downloads_renamed\2024-12-29_051013.mp4
Saved: snapchat_downloads_renamed\2024-12-27_052151.zip
Saved: snapchat_downloads_renamed\2024-12-26_152705.zip
Saved: snapchat_downloads_renamed\2024-12-25_112418.zip
Saved: snapchat_downloads_renamed\2024-12-25_112418.mp4
Saved: snapchat_downloads_renamed\2024-12-25_013803.jpg
Saved: snapchat_downloads_renamed\2024-12-23_115119.mp4
Saved: snapchat_downloads_renamed\2024-12-23_060704.jpg
Saved: snapchat_downloads_renamed\2024-12-22_091330.zip
Saved: snapchat_downloads_renamed\2024-12-22_085311.zip
Saved: snapchat_downloads_renamed\2024-12-22_085211.zip
Saved: snapchat_downloads_renamed\2024-12-22_085124.zip
Saved: snapchat_downloads_renamed\2024-12-22_084220.zip
Saved: snapchat_downloads_renamed\2024-12-19_104242.jpg
Saved: snapchat_downloads_renamed\2024-12-17_045025.mp4
Saved: snapchat_downloads_renamed\2024-12-14_114653.zip
Saved: snapchat_downloads_renamed\2024-12-14_114621.mp4
Saved: snapchat_downloads_renamed\2024-12-14_113

Saved: snapchat_downloads_renamed\2024-10-30_011947.mp4
Saved: snapchat_downloads_renamed\2024-10-29_235125.zip
Saved: snapchat_downloads_renamed\2024-10-29_235016.mp4
Saved: snapchat_downloads_renamed\2024-10-29_125429.zip
Saved: snapchat_downloads_renamed\2024-10-29_113345.zip
Saved: snapchat_downloads_renamed\2024-10-29_021756.jpg
Saved: snapchat_downloads_renamed\2024-10-29_021713.zip
Saved: snapchat_downloads_renamed\2024-10-29_010757.zip
Saved: snapchat_downloads_renamed\2024-10-28_222736.zip
Saved: snapchat_downloads_renamed\2024-10-28_181537.zip
Saved: snapchat_downloads_renamed\2024-10-28_133330.zip
Saved: snapchat_downloads_renamed\2024-10-28_132238.zip
Saved: snapchat_downloads_renamed\2024-10-28_130637.zip
Saved: snapchat_downloads_renamed\2024-10-28_120951.zip
Saved: snapchat_downloads_renamed\2024-10-28_120804.zip
Saved: snapchat_downloads_renamed\2024-10-28_113112.mp4
Saved: snapchat_downloads_renamed\2024-10-27_221808.zip
Saved: snapchat_downloads_renamed\2024-10-27_163

Saved: snapchat_downloads_renamed\2024-09-14_095155.jpg
Saved: snapchat_downloads_renamed\2024-09-14_094927.jpg
Saved: snapchat_downloads_renamed\2024-09-14_034806.mp4
Saved: snapchat_downloads_renamed\2024-09-14_015754.zip
Saved: snapchat_downloads_renamed\2024-09-14_002722.zip
Saved: snapchat_downloads_renamed\2024-09-13_235613.zip
Saved: snapchat_downloads_renamed\2024-09-13_233803.zip
Saved: snapchat_downloads_renamed\2024-09-13_154252.zip
Saved: snapchat_downloads_renamed\2024-09-13_124655.mp4
Saved: snapchat_downloads_renamed\2024-09-13_124649.jpg
Saved: snapchat_downloads_renamed\2024-09-13_124622.mp4
Saved: snapchat_downloads_renamed\2024-09-13_124454.mp4
Saved: snapchat_downloads_renamed\2024-09-12_180409.zip
Saved: snapchat_downloads_renamed\2024-09-11_193514.mp4
Saved: snapchat_downloads_renamed\2024-09-11_153739.zip
Saved: snapchat_downloads_renamed\2024-09-10_213227.zip
Saved: snapchat_downloads_renamed\2024-09-10_084349.zip
Saved: snapchat_downloads_renamed\2024-09-09_193

Saved: snapchat_downloads_renamed\2024-08-09_181435.jpg
Saved: snapchat_downloads_renamed\2024-08-09_163905.mp4
Saved: snapchat_downloads_renamed\2024-08-09_163817.jpg
Saved: snapchat_downloads_renamed\2024-08-07_175240.jpg
Saved: snapchat_downloads_renamed\2024-08-06_214412.jpg
Saved: snapchat_downloads_renamed\2024-08-06_131636.jpg
Saved: snapchat_downloads_renamed\2024-08-06_093636.zip
Saved: snapchat_downloads_renamed\2024-08-05_165018.mp4
Saved: snapchat_downloads_renamed\2024-08-05_012850.mp4
Saved: snapchat_downloads_renamed\2024-08-05_003143.zip
Saved: snapchat_downloads_renamed\2024-08-04_183347.mp4
Saved: snapchat_downloads_renamed\2024-08-03_225952.mp4
Saved: snapchat_downloads_renamed\2024-08-03_140646.jpg
Saved: snapchat_downloads_renamed\2024-08-03_025542.zip
Saved: snapchat_downloads_renamed\2024-08-03_021211.jpg
Saved: snapchat_downloads_renamed\2024-08-02_203308.mp4
Saved: snapchat_downloads_renamed\2024-08-02_132238.jpg
Saved: snapchat_downloads_renamed\2024-08-02_132

Saved: snapchat_downloads_renamed\2024-07-17_121119.jpg
Saved: snapchat_downloads_renamed\2024-07-17_115850.jpg
Saved: snapchat_downloads_renamed\2024-07-17_115849.zip
Saved: snapchat_downloads_renamed\2024-07-17_083734.jpg
Saved: snapchat_downloads_renamed\2024-07-15_140121.jpg
Saved: snapchat_downloads_renamed\2024-07-15_135049.zip
Saved: snapchat_downloads_renamed\2024-07-15_133901.mp4
Saved: snapchat_downloads_renamed\2024-07-15_123757.jpg
Saved: snapchat_downloads_renamed\2024-07-15_123745.jpg
Saved: snapchat_downloads_renamed\2024-07-15_090444.zip
Saved: snapchat_downloads_renamed\2024-07-14_201321.jpg
Saved: snapchat_downloads_renamed\2024-07-14_100352.jpg
Saved: snapchat_downloads_renamed\2024-07-14_092522.zip
Saved: snapchat_downloads_renamed\2024-07-14_061258.zip
Saved: snapchat_downloads_renamed\2024-07-13_114343.jpg
Saved: snapchat_downloads_renamed\2024-07-12_164447.jpg
Saved: snapchat_downloads_renamed\2024-07-12_154054.mp4
Saved: snapchat_downloads_renamed\2024-07-12_095

Saved: snapchat_downloads_renamed\2024-05-29_173625.mp4
Saved: snapchat_downloads_renamed\2024-05-29_173605.mp4
Saved: snapchat_downloads_renamed\2024-05-29_173238.mp4
Saved: snapchat_downloads_renamed\2024-05-29_173232.jpg
Saved: snapchat_downloads_renamed\2024-05-28_140410.zip
Saved: snapchat_downloads_renamed\2024-05-27_195004.mp4
Saved: snapchat_downloads_renamed\2024-05-27_194832.mp4
Saved: snapchat_downloads_renamed\2024-05-27_194404.mp4
Saved: snapchat_downloads_renamed\2024-05-27_114529.zip
Saved: snapchat_downloads_renamed\2024-05-26_171352.mp4
Saved: snapchat_downloads_renamed\2024-05-26_171303.mp4
Saved: snapchat_downloads_renamed\2024-05-26_171243.mp4
Saved: snapchat_downloads_renamed\2024-05-26_164748.mp4
Saved: snapchat_downloads_renamed\2024-05-26_155033.jpg
Saved: snapchat_downloads_renamed\2024-05-26_150722.zip
Saved: snapchat_downloads_renamed\2024-05-26_145545.jpg
Saved: snapchat_downloads_renamed\2024-05-26_135254.jpg
Saved: snapchat_downloads_renamed\2024-05-25_224

Saved: snapchat_downloads_renamed\2024-04-20_042640.zip
Saved: snapchat_downloads_renamed\2024-04-20_042543.zip
Saved: snapchat_downloads_renamed\2024-04-19_132109.mp4
Saved: snapchat_downloads_renamed\2024-04-19_131711.mp4
Saved: snapchat_downloads_renamed\2024-04-18_190147.zip
Saved: snapchat_downloads_renamed\2024-04-18_184128.zip
Saved: snapchat_downloads_renamed\2024-04-18_091146.mp4
Saved: snapchat_downloads_renamed\2024-04-17_213121.zip
Saved: snapchat_downloads_renamed\2024-04-17_211939.mp4
Saved: snapchat_downloads_renamed\2024-04-17_165020.zip
Saved: snapchat_downloads_renamed\2024-04-17_164536.zip
Saved: snapchat_downloads_renamed\2024-04-16_122223.mp4
Saved: snapchat_downloads_renamed\2024-04-16_122125.mp4
Saved: snapchat_downloads_renamed\2024-04-14_214117.zip
Saved: snapchat_downloads_renamed\2024-04-12_193119.zip
Saved: snapchat_downloads_renamed\2024-04-12_193059.mp4
Saved: snapchat_downloads_renamed\2024-04-12_192209.mp4
Saved: snapchat_downloads_renamed\2024-04-12_081

Saved: snapchat_downloads_renamed\2024-02-28_193315.jpg
Saved: snapchat_downloads_renamed\2024-02-28_191902.mp4
Saved: snapchat_downloads_renamed\2024-02-28_191825.mp4
Saved: snapchat_downloads_renamed\2024-02-27_221132.mp4
Saved: snapchat_downloads_renamed\2024-02-27_220742.mp4
Saved: snapchat_downloads_renamed\2024-02-27_220120.jpg
Saved: snapchat_downloads_renamed\2024-02-27_215737.zip
Saved: snapchat_downloads_renamed\2024-02-27_172909.zip
Saved: snapchat_downloads_renamed\2024-02-25_185442.mp4
Saved: snapchat_downloads_renamed\2024-02-25_185318.mp4
Saved: snapchat_downloads_renamed\2024-02-24_144011.jpg
Saved: snapchat_downloads_renamed\2024-02-24_141003.jpg
Saved: snapchat_downloads_renamed\2024-02-24_131240.jpg
Saved: snapchat_downloads_renamed\2024-02-21_004303.mp4
Saved: snapchat_downloads_renamed\2024-02-20_234038.mp4
Saved: snapchat_downloads_renamed\2024-02-20_220628.mp4
Saved: snapchat_downloads_renamed\2024-02-20_215722.mp4
Saved: snapchat_downloads_renamed\2024-02-20_213

In [7]:
# Check for duplicate dates to ensure no memory will be overwritten
dates = []
for item in memories:
    dates.append(item["Date"])

dates_dict = {}
for date in dates:
    if date not in dates_dict:
        dates_dict[date]=1
    else:
        dates_dict[date]+=1
        print(date)



2024-12-25 11:24:18 UTC
2024-12-06 19:59:10 UTC
2024-11-07 01:42:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-23 16:05:27 UTC
2024-10-02 20:39:15 UTC
2024-05-09 19:37:20 UTC


In [ ]:
# to do

# check for duplicates at start of file (maybe computationally expensive? rather than going one by one.. 
# compare doing it within the for loop vs ahead of it)

# see how complicated it would be to extract a zip file and label them _overlay or base or something 
# or see how many files are in each zip file and investigate if there's ever more than 2


In [8]:
dupes = {}

for item in memories:
    
    if dates_dict[item["Date"]] > 1:
        
        url = item["Media Download Url"]
        media_type = item["Media Type"]

        # clean date for filename
        date = datetime.strptime(item["Date"], "%Y-%m-%d %H:%M:%S UTC")
        if date in dupes:
            dupes[date]+=1
        else:
            dupes[date]=1
        base_name = date.strftime("%Y-%m-%d_%H%M%S") + "_" + str(dupes[date])

        # not downloading the same files we already downloaded before failure

        try:
            response = requests.get(url, allow_redirects=True)
            response.raise_for_status()
        except:
            if response.status_code >= 500:
                time.sleep(5)
                try:
                    response = requests.get(url, allow_redirects=True)
                    response.raise_for_status()
                except:
                    print("Could not save: ", filename)


        ext = extension_from_headers(response.headers, media_type)
        filename = os.path.join(output_dir, base_name + ext)

        with open(filename, "wb") as f:
            f.write(response.content)

        print("Saved:", filename)


Saved: snapchat_downloads_renamed\2024-12-25_112418_1.zip
Saved: snapchat_downloads_renamed\2024-12-25_112418_2.mp4
Saved: snapchat_downloads_renamed\2024-12-06_195910_1.jpg
Saved: snapchat_downloads_renamed\2024-12-06_195910_2.jpg
Saved: snapchat_downloads_renamed\2024-11-07_014227_1.zip
Saved: snapchat_downloads_renamed\2024-11-07_014227_2.jpg
Saved: snapchat_downloads_renamed\2024-10-23_160527_1.mp4
Saved: snapchat_downloads_renamed\2024-10-23_160527_2.mp4
Saved: snapchat_downloads_renamed\2024-10-23_160527_3.mp4
Saved: snapchat_downloads_renamed\2024-10-23_160527_4.mp4
Saved: snapchat_downloads_renamed\2024-10-23_160527_5.mp4
Saved: snapchat_downloads_renamed\2024-10-23_160527_6.mp4
Saved: snapchat_downloads_renamed\2024-10-23_160527_7.mp4
Saved: snapchat_downloads_renamed\2024-10-23_160527_8.mp4
Saved: snapchat_downloads_renamed\2024-10-23_160527_9.mp4
Saved: snapchat_downloads_renamed\2024-10-23_160527_10.mp4
Saved: snapchat_downloads_renamed\2024-10-23_160527_11.mp4
Saved: snapc